In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:8"

import torch
import torchvision
import numpy as np
import random
import pickle

import torch.nn as nn
from sklearn.metrics import accuracy_score, f1_score
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset


/home/dilan/anaconda3/envs/text_tag_combined_exp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from helper_functions import *
from model_component import *
from parameters import *

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
dataset = PuzzleImageDataset(img_dir='./dataset/train', data_file=None, label_file_name='label_train.txt', grid_size=6)
dataset.preprocess(device, "train_dataset_full")

Preprocessing of data started!
10 completed.
20 completed.
30 completed.
40 completed.
50 completed.
60 completed.
70 completed.
80 completed.
90 completed.
100 completed.
110 completed.
120 completed.
130 completed.
140 completed.
150 completed.
160 completed.
170 completed.
180 completed.
190 completed.
200 completed.
210 completed.
220 completed.
230 completed.
240 completed.
250 completed.
260 completed.
270 completed.
280 completed.
290 completed.
300 completed.
310 completed.
320 completed.
330 completed.
340 completed.
350 completed.
360 completed.
370 completed.
380 completed.
390 completed.
400 completed.
410 completed.
420 completed.
430 completed.
440 completed.
450 completed.
460 completed.
470 completed.
480 completed.
490 completed.
500 completed.
510 completed.
520 completed.
530 completed.
540 completed.
550 completed.
560 completed.
570 completed.
580 completed.
590 completed.
600 completed.
610 completed.
620 completed.
630 completed.
640 completed.
650 completed.
660

In [6]:
def custom_loss(log_probs, one_hot_positions):

    required_pos_loss = nn.functional.cross_entropy(log_probs, one_hot_positions)

    return required_pos_loss

In [7]:
from model_component import *
from parameters import *


for seed_no in range(0, 20):
    # Setting the random seed value for model
    seed_val = random.randint(0, 1000)

    torch.use_deterministic_algorithms(False)
    np.random.seed(seed_val)
    random.seed(seed_val)
    torch.manual_seed(seed_val)

    dataset = PuzzleImageDataset(img_dir='./dataset/train', data_file="train_dataset_full.pkl", label_file_name='label_train.txt', grid_size=6)

    train_dataset, test_dataset, validation_dataset = torch.utils.data.random_split(dataset, [0.7, 0.15, 0.15])

    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)

    model = PermutationPredictor(num_heads=NUM_HEADS, dropout=DROP_OUT)
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.9, patience=30, threshold=1e-4, verbose=True)

    for epoch in range(EPOCHS):
        print(f"\nTraining of Epoch {epoch+1} started!\n")
        
        true_lbls = []
        predicted_lbls = []
        for i, data in enumerate(iter(train_dataloader), 0):
            
            optimizer.zero_grad()
            
            img_vecs, img_grid_vec, similarity_features, lbls = data
            similarity_features = similarity_features.to(torch.float32).to(device)
            img_vecs = img_vecs.to(torch.float32).to(device)
            
            output = model(img_vecs, similarity_features)
    
            p_lbls = torch.argmax(nn.functional.softmax(output, dim=-1), dim=-1).tolist()
            t_lbls = lbls.tolist()

            true_lbls += t_lbls
            predicted_lbls += p_lbls

            one_hot_lbls = nn.functional.one_hot(lbls, num_classes=50)
            loss = custom_loss(output, one_hot_lbls.type(torch.FloatTensor).to(device))

            print(f"Loss at {i} of Epoch {epoch+1}: {float(loss)}")
            
            loss.backward()
            optimizer.step()
            scheduler.step(loss)
        
        print(f"Candidate {seed_no} => Epoch {epoch+1} Training accuracy: {accuracy_score(true_lbls, predicted_lbls)*100}%")
        print(f"Candidate {seed_no} => Epoch {epoch+1} Training f1 score: {f1_score(true_lbls, predicted_lbls, average='macro')*100}%")
        print(f"Candidate {seed_no} => Epoch {epoch+1} Training f1 Class Level: \n{f1_score(true_lbls, predicted_lbls, average=None)*100}%")

        model.eval()
        with torch.no_grad():

            true_lbls = []
            predicted_lbls = []
            
            print(f"\nEvaluation of Epoch {epoch+1} started!")
            for i, data in enumerate(iter(validation_dataloader), 0):
                
                img_vecs, img_grid_vec, similarity_features, lbls = data
                similarity_features = similarity_features.to(torch.float32).to(device)
                img_vecs = img_vecs.to(torch.float32).to(device)
                
                output = model(img_vecs, similarity_features)

                p_lbls = torch.argmax(nn.functional.softmax(output, dim=-1), dim=-1).tolist()
                t_lbls = lbls.tolist()

                true_lbls += t_lbls
                predicted_lbls += p_lbls

                one_hot_lbls = nn.functional.one_hot(lbls, num_classes=50)
                loss = custom_loss(output, one_hot_lbls.type(torch.FloatTensor).to(device))

                print(f"Candidate {seed_no} => Validation Loss at {i} of Epoch {epoch+1}: {float(loss)}")

            print(f"Candidate {seed_no} => Epoch {epoch+1} Validation accuracy: {accuracy_score(true_lbls, predicted_lbls)*100}%")
            print(f"Candidate {seed_no} => Epoch {epoch+1} Validation f1 score: {f1_score(true_lbls, predicted_lbls, average='macro')*100}%")
            print(f"Candidate {seed_no} => Epoch {epoch+1} Validation f1 Class Level: \n{f1_score(true_lbls, predicted_lbls, average=None)*100}%")

        model.train()
    torch.save(model.state_dict(), f'ensemble_prediction/puzzle_classifier_{seed_no}.cpt')


Training of Epoch 1 started!

Loss at 0 of Epoch 1: 3.8998618125915527
Loss at 1 of Epoch 1: 4.043671131134033
Loss at 2 of Epoch 1: 4.072044849395752
Loss at 3 of Epoch 1: 3.9314255714416504
Loss at 4 of Epoch 1: 4.014239311218262
Loss at 5 of Epoch 1: 4.07363224029541
Loss at 6 of Epoch 1: 4.098950386047363
Loss at 7 of Epoch 1: 4.10112190246582
Loss at 8 of Epoch 1: 3.9745326042175293
Loss at 9 of Epoch 1: 3.9035162925720215
Loss at 10 of Epoch 1: 4.045330047607422
Loss at 11 of Epoch 1: 3.9771227836608887
Loss at 12 of Epoch 1: 3.957180976867676
Loss at 13 of Epoch 1: 3.9587161540985107
Loss at 14 of Epoch 1: 3.9574077129364014
Loss at 15 of Epoch 1: 3.9786062240600586
Loss at 16 of Epoch 1: 3.9932518005371094
Loss at 17 of Epoch 1: 3.9113352298736572
Loss at 18 of Epoch 1: 3.954681873321533
Loss at 19 of Epoch 1: 3.953143358230591
Loss at 20 of Epoch 1: 3.9132847785949707
Loss at 21 of Epoch 1: 3.9675445556640625
Loss at 22 of Epoch 1: 3.889963388442993
Loss at 23 of Epoch 1: 3.9

## Evaluation

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from helper_functions import *
from model_component import *
from parameters import *
from os import listdir
from os.path import isfile, join

models_dir = "ensemble_prediction"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model_files = [f for f in listdir(models_dir) if (isfile(join(models_dir, f)) and f.endswith('cpt'))]
models = []

for model_name in model_files:

    model = PermutationPredictor(num_heads=NUM_HEADS, dropout=DROP_OUT)
    model.to(device)
    model.load_state_dict(torch.load(join(models_dir, model_name)))
    model.eval()

    models.append(model)

len(models)

20

In [10]:
test_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

true_lbls = []
predicted_lbls = []
for i, data in enumerate(iter(test_dataloader), 0):
    
    img_vecs, img_grid_vec, similarity_features, lbls = data
    similarity_features = similarity_features.to(torch.float32).to(device)
    img_vecs = img_vecs.to(torch.float32).to(device)
    
    output = None
    for model in models:
        if(output is None):
            temp = model(img_vecs, similarity_features)
            arg_maxes = torch.argmax(nn.functional.softmax(temp, dim=-1), dim=-1)
            output = nn.functional.one_hot(arg_maxes, num_classes=50)
            
        else:
            temp = model(img_vecs, similarity_features)
            arg_maxes = torch.argmax(nn.functional.softmax(temp, dim=-1), dim=-1)
            output += nn.functional.one_hot(arg_maxes, num_classes=50)
    
    output = output/len(models)

    p_lbls = torch.argmax(nn.functional.softmax(output, dim=-1), dim=-1).tolist()
    t_lbls = lbls.tolist()

    true_lbls += t_lbls
    predicted_lbls += p_lbls
    
    one_hot_lbls = nn.functional.one_hot(lbls, num_classes=50)
    loss = custom_loss(output, one_hot_lbls.type(torch.FloatTensor).to(device))
    
    print(f"Test loss for minibatch {i+1}: {loss}")

print(f"Test accuracy: {accuracy_score(true_lbls, predicted_lbls)*100}%")
print(f"Test f1 score: {f1_score(true_lbls, predicted_lbls, average='macro')*100}%")
print(f"Test f1 Class Level: \n{f1_score(true_lbls, predicted_lbls, average=None)*100}%")

Test loss for minibatch 1: 2.961031913757324
Test loss for minibatch 2: 2.964066982269287
Test loss for minibatch 3: 2.963352680206299
Test loss for minibatch 4: 2.960235118865967
Test loss for minibatch 5: 2.967072010040283
Test loss for minibatch 6: 2.960264205932617
Test loss for minibatch 7: 2.9610328674316406
Test loss for minibatch 8: 2.960280418395996
Test loss for minibatch 9: 2.961744785308838
Test loss for minibatch 10: 2.9655556678771973
Test loss for minibatch 11: 2.947324752807617
Test loss for minibatch 12: 2.9655752182006836
Test loss for minibatch 13: 2.9541523456573486
Test loss for minibatch 14: 2.9663405418395996
Test loss for minibatch 15: 2.962528705596924
Test loss for minibatch 16: 2.980917453765869
Test loss for minibatch 17: 2.964794635772705
Test loss for minibatch 18: 2.956465244293213
Test loss for minibatch 19: 2.9526281356811523
Test loss for minibatch 20: 2.955676794052124
Test loss for minibatch 21: 2.9572250843048096
Test loss for minibatch 22: 2.965582

In [11]:
# import onnx
# input_names = ["image_vectors", "similarity_features"]
# output_names = ["Permutation Prediction"]

# torch.onnx.export(models[0], args=(img_vecs, similarity_features), f="model.onnx", input_names=input_names, output_names=output_names)